In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline 

from sklearn.ensemble import RandomForestClassifier        
from sklearn.linear_model import LogisticRegression, \
                                 SGDClassifier 
# import xgboost as xgb
    

# Instantiate models
logit = LogisticRegression(penalty='l1', n_jobs=1)
rfm = RandomForestClassifier(n_jobs=1)
sgd = SGDClassifier(n_jobs=1)

In [6]:
datapath = '/Users/daniellee/Desktop/Data/Datasets/talkingdata_fraud/'
train_X = pd.read_hdf(datapath+'m3_features/a3_train.hdf')
train_y = pd.read_pickle(datapath+'train_y_int8')

In [4]:
train_y.dtypes

is_attributed    uint8
dtype: object

In [7]:
train_ip = train_X['ip']
train_X = train_X.drop('ip', axis=1)

# test_ip = test_X['ip']
# test_X = test_X.drop('ip', axis=1)

In [ ]:
# if set(test_X.columns) == set(train_X.columns):
    
from sklearn.externals import joblib

# Train Models
logit.fit(train_X, train_y)
rfm.fit(train_X, train_y)
sgd.fit(train_X, train_y)

# Persist models
persist_loc = '/Users/daniellee/Dropbox/Jupyter/Kaggle/talkingdata_fraud/model_persists/'
mod_name_comb = [[logit, rfm, sgd], ['logit', 'rfm', 'xgd']]
for mod in zip(*mod_name_comb):
    joblib.dumps(mod, 'a3_model_{}.pk1'.format(mod[1]))

In [ ]:
# Predict probability of attribution for each model

test_X = pd.read_pickle(datapath+'m3_features/a3_test')

def model_predict(mod, name, test):
    print('Predicting using {}'.format(name))
    pred_prob = mod.predict_proba(test)
    print('Prediction complete...\n')
    return pred_prob

logit_pred, rfm_pred, sgd_pred = (
                    [model_predict(mod, name, test_X) for mod, name in zip(*mod_name_comb)]
)

# Create submission file per each model prediction 
submission = pd.read_pickle(datapath+'submission_reduced.csv')

for pred, name in zip([logit_pred, rfm_pred, sgd_pred],['logit', 'rfm', 'xgd']):
    submission['is_attributed'] = pred
    submission.to_csv(datafile'submissions/a3_submission_{}.csv'.format(name))